In [1]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
df = pd.read_csv('../data/myPersonality/mypersonality_final.csv', encoding='latin1')
print("The size of data is {0}".format(df.shape[0]))
labels = ['cEXT','cNEU','cAGR','cCON','cOPN']
df.head()



The size of data is 9917


,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [3]:
x = df['STATUS']
y = df[labels[0]].values
y = (y == 'y').astype(int)

In [4]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(x, y, 
                                                            random_state=2018, test_size=0.1)

In [16]:

train_df = pd.DataFrame({
    'text': train_inputs,
    'label':train_labels
})
print(Counter(train_df['label'].values))

df_yes = train_df[train_df['label'] == 1] 
df_no = train_df[train_df['label'] == 0] 
df_no = df_no.sample(n=len(df_yes))
train_df = df_no.append(df_yes)
train_df = train_df.sample(frac=1, random_state = 24).reset_index(drop=True)
print(Counter(train_df['label'].values))

eval_df = pd.DataFrame({
    'text': validation_inputs,
    'label': validation_labels
})
print(Counter(eval_df['label'].values))
print(eval_df.head())



Counter({0: 5136, 1: 3789})
Counter({1: 3789, 0: 3789})
Counter({0: 571, 1: 421})
                                                   text  label
2452  just experienced orthostatic hypotension. trippy.      1
1923  likes pina coladas and getting caught in the rain      0
8394                                       is a new man      1
1020  If you stick with a Rubik's Cube long enough, ...      0
2424  day of phone dates with trappers & furbearer b...      1


In [18]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=True, cuda_device=0) # You can set class weights by using the optional weight argument



In [19]:
# Train the model
model.train_model(train_df, args={'overwrite_output_dir':True})



Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.603049

Training of roberta model complete. Saved to outputs/.


In [25]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Converting to features started. Cache is not used.


In [26]:
result

{'mcc': 0.0,
 'tp': 0,
 'tn': 571,
 'fp': 0,
 'fn': 421,
 'eval_loss': 0.6888120217669395}

In [27]:
len(model_outputs)

992

In [24]:
len(wrong_predictions)

421